# Hilbert行列の条件数の数値計算

## Hilbert行列の逆行列
$n$次Hilbert行列の逆行列の第$(i,j)$成分は
$$ 
     (-1)^{i+j}(i+j-1)\begin{pmatrix} n+i-1 \\ n-j \end{pmatrix}
    \begin{pmatrix} n+j-1 \\ n-i \end{pmatrix} 
    \begin{pmatrix} i+j-2 \\ i-1 \end{pmatrix}^2
$$
であることが示されている．念のためJuliaで計算して確認しておこう．

In [2]:
setprecision(1000)
H(n) = [big(1)/(i+j-1) for i in 1:n, j in 1:n]    
Hinv(n) = [(-1)^(i+j)*(i+j-1)*binomial(big(n+i-1),n-j)*binomial(big(n+j-1),n-i)*binomial(big(i+j-2),i-1)^2 
        for i in 1:n, j in 1:n]
H(5)*Hinv(5) 

5×5 Matrix{BigFloat}:
 1.0  0.0  0.0  0.0           0.0
 0.0  1.0  0.0  0.0           0.0
 0.0  0.0  1.0  3.05812e-297  0.0
 0.0  0.0  0.0  1.0           0.0
 0.0  0.0  0.0  0.0           1.0

## 1-normに関する条件数の計算
1-normから誘導される行列ノルムは，「最大列和」であったから，そのまま実装する：

In [4]:
mat1norm(A) = maximum([sum(abs.(A[:,j])) for j in 1:size(A)[1]]) 

mat1norm (generic function with 1 method)

`sum(abs.(A[:,j])`は$A$の第$j$列の絶対値の和を計算している．
`mat1norm(H(n))`を計算した結果下記の表のとおり．

In [11]:
for n in 1:20
    @show Float64(mat1norm(H(n))*mat1norm(Hinv(n)))
end

Float64(mat1norm(H(n)) * mat1norm(Hinv(n))) = 1.0
Float64(mat1norm(H(n)) * mat1norm(Hinv(n))) = 27.0
Float64(mat1norm(H(n)) * mat1norm(Hinv(n))) = 748.0
Float64(mat1norm(H(n)) * mat1norm(Hinv(n))) = 28375.0
Float64(mat1norm(H(n)) * mat1norm(Hinv(n))) = 943656.0
Float64(mat1norm(H(n)) * mat1norm(Hinv(n))) = 2.9070279e7
Float64(mat1norm(H(n)) * mat1norm(Hinv(n))) = 9.851948865e8
Float64(mat1norm(H(n)) * mat1norm(Hinv(n))) = 3.3872791095e10
Float64(mat1norm(H(n)) * mat1norm(Hinv(n))) = 1.0996545413425e12
Float64(mat1norm(H(n)) * mat1norm(Hinv(n))) = 3.5357439251992e13
Float64(mat1norm(H(n)) * mat1norm(Hinv(n))) = 1.2337023575988502e15
Float64(mat1norm(H(n)) * mat1norm(Hinv(n))) = 4.115445402289639e16
Float64(mat1norm(H(n)) * mat1norm(Hinv(n))) = 1.324409009034709e18
Float64(mat1norm(H(n)) * mat1norm(Hinv(n))) = 4.537757843943819e19
Float64(mat1norm(H(n)) * mat1norm(Hinv(n))) = 1.5391915629553123e21
Float64(mat1norm(H(n)) * mat1norm(Hinv(n))) = 5.0627747875083205e22
Float64(mat1norm(H(n)) 

## Remark

- Julia言語では，行列 $p$-normはLinearAlgebraパッケージの`opnorm(A,p)`で計算できる．
また，$\cond_p(A)$は 同パッケージの`cond(A,p)`で計算できる．
しかし，いずれの関数も `Float64`型行列にのみ対応していて，
`BigFloat`型行列を引数として渡すとエラーになってしまう．
- 条件数の大きな行列$A$の逆行列を`inv(A)`などで計算した場合，一般に誤差が大きいので注意．
例として，Hilbert行列で試してみよう．

In [12]:
H(n) = [1/(i+j-1) for i in 1:n, j in 1:n]
inv(H(15))[15,15]   # (15,15)成分


4.391966739292475e13

しかし，$H_{15}^{-1}$の$(15,15)$成分の真値は $46670906271240000 \approx$ `4.666e17` であり，
指数ですら全く異なる値になってしまっている．
 
- $A$の条件数が大きい場合，`inv(A)`と同様に，`cond(A)`の値の信頼性が低いことに注意．

In [14]:
using LinearAlgebra
cond(H(15),1)

1.221413576195944e18

真値は先に計算したとおり，約`1.5391915629553123e+21`なので，全然違う値になってしまった．

- 与えられた行列の条件数の正確な値を計算することは難しいが，
おおよその値を（`cond(A)`より正確に）推定する方法は既に知られている．・・・齊藤「数値解析」（共立出版）のp.180 参照

正確な値でなくとも，条件数の大小が判明するだけでも大きな役に立つ．

- 1-normの推定法として，Hagerのアルゴリズム
[Hager: Condition Estimates,  SIAM J. Sci. Stat. Comput. 5  (1984) 311-316.]
が最も優れているらしい．
LAPACKやMATLABの`condest()`の実装（の一部）に用いられている．

- LAPACKは線形代数の数値計算ライブラリである（公式サイト: [LAPACK -- Linear Algebra PACKage](\url{https://netlib.org/lapack})）.「レイパック」や「エルエーパック」と発音するのが正しいとされるが，それを知っていてもなお「ラパック」と呼んでしまう．
